In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import seaborn as sns

import os
import random

from PIL import Image


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torchvision import datasets, transforms
from utils import *
import albumentations as album
#import extractors

sns.set(style='white', context='notebook', palette='deep')

In [2]:
if torch.cuda.is_available():  
    dev = "cuda:0" 
else:  
    dev = "cpu"  
device = torch.device(dev)  

In [3]:
# def keep_image_size_open(path, size=(512, 512)):
#     img = Image.open(path)
#     side = max(img.size)  # Get the longest side of the image
#     mask = Image.new('RGB', (side, side), (0, 0, 0))  # Create a square canvas
#     mask.paste(img, (0, 0))  # Paste the original image on the left top of the canvas
#     mask = mask.resize(size)  # Resize the new image to a uniform size
#     return mask

# def keep_mask_size_open(path, size=(512, 512)):
#     img = Image.open(path)
#     side = max(img.size)  # Get the longest side of the image
#     mask = Image.new('L', (side, side), 0)  # Create a square canvas
#     mask.paste(img, (0, 0))  # Paste the original image on the left top of the canvas
#     mask = mask.resize(size)  # Resize the new image to a uniform size
#     return mask

In [4]:
# Need to be run only one time
ENCODER = 'resnext50_32x4d'
ENCODER_WEIGHTS = 'imagenet'
CLASSES = 27
ACTIVATION = 'sigmoid' # could be None for logits or 'softmax2d','softmax' for multiclass segmentation

# create segmentation model with pretrained encoder
model = smp.PSPNet(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=27, 
    activation=ACTIVATION,
)

preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

NameError: name 'smp' is not defined

In [8]:
# Convert a PIL Image to tensor.
transform = transforms.Compose([transforms.ToTensor()])
class MyDataset(Dataset):
    def __init__(self, path):
        self.path = path
        self.name = os.listdir(os.path.join(path, 'train_masks'))

    def __len__(self):
        return len(self.name)

    def __getitem__(self, index):
        msk_name = self.name[index]
        msk_path = os.path.join(self.path, 'train_masks', msk_name)
        img_path = os.path.join(self.path, 'train_images', msk_name.replace('png', 'jpg'))
        img = keep_image_size_open(img_path)
        mask = keep_mask_size_open(msk_path)
        img = transform(img) # this take score of shape that is channel first, normalise and covert to tensor
        mask = np.array(mask)
        mask = torch.from_numpy(mask).to(torch.int64)
        mask = F.one_hot(mask, num_classes=27)
        mask = torch.transpose(mask,2,0)
        return img,mask


if __name__ == '__main__':
    data = MyDataset('')
    check_image = data[100][0] # checking for the random 100th image
    check_mask = data[10|0][1]
    print(check_image.shape,check_mask.shape)
    print(check_image.dtype,check_mask.dtype)
    print(len(data))

torch.Size([3, 512, 512]) torch.Size([27, 512, 512])
torch.float32 torch.int64
261


In [20]:
# i1 = Image.open(os.path.join('train_images',os.listdir('train_images')[143]))
# m1 = Image.open(os.path.join('train_masks',os.listdir('train_masks')[143]))

# plt.subplot(1,2,1)
# plt.imshow(i1)
# plt.subplot(1,2,2)
# plt.imshow(m1)
# plt.show()

In [24]:
# converting the mask into one hot

In [9]:
bs = 5
nw = 0
# Splitting into Train and Val
full_dataset = MyDataset('')
train_size = int(0.8 * len(full_dataset))
val_size   = len(full_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

# Creating  data_loader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=bs,num_workers=nw,shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=bs,num_workers=nw,shuffle=True)


In [10]:
it, lt = next(iter(train_loader))
print(it.shape,lt.shape)

iv, lv = next(iter(val_loader))
print(iv.dtype,lv.dtype)

torch.Size([5, 3, 512, 512]) torch.Size([5, 27, 512, 512])
torch.float32 torch.int64


In [11]:
#getting the pspnet model

In [13]:
len(train_loader) # because of bs =5

42